In [1]:
import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .getOrCreate()

print("APP Name :"+ spark.sparkContext.appName);
print("Master :"+ spark.sparkContext.master);

APP Name :GenericAppName
Master :local[*]


In [2]:
# col_names = ["tcp_flags", "tcp_time_delta", "tcp_len", "mqtt_conack_flags", "mqtt_conack_flags_reserved", "mqtt_conack_flags_sp",
#              "mqtt_conack_val", "mqtt_conflag_cleansess", "mqtt_conflag_passwd", "mqtt_conflag_qos", "mqtt.conflag.reserved", "mqtt_conflag_retain",
#              "mqtt_conflag_uname", "mqtt_conflag_willflag", "mqtt_conflags", "mqtt_dupflag", "mqtt_hdrflags", "mqtt_kalive",
#              "mqtt_len, mqtt_msg", "mqtt_msgid", "mqtt_msgtype", "mqtt_proto_len", "mqtt_protoname", "mqtt_qos", "mqtt_retain",
#              "mqtt_sub_qos", "mqtt_suback_qos", "mqtt_ver", "mqtt_willmsg", "mqtt_willmsg_len", "mqtt_willtopic", "mqtt_willtopic_len",
#              "target"]
col_names = ["tcp_flags", "tcp_time_delta", "tcp_len", "mqtt_conack_flags", "mqtt_conack_flags_reserved", "mqtt_conack_flags_sp", 
             "mqtt_conack_val", "mqtt_conflag_cleansess", "mqtt_conflag_passwd", "mqtt_conflag_qos", "mqtt_conflag_reserved", "mqtt_conflag_retain", 
             "mqtt_conflag_uname", "mqtt_conflag_willflag", "mqtt_conflags", "mqtt_dupflag", "mqtt_hdrflags", "mqtt_kalive",
              "mqtt_len", "mqtt_msg", "mqtt_msgid", "mqtt_msgtype", "mqtt_proto_len", "mqtt_protoname", "mqtt_qos", "mqtt_retain",
              "mqtt_sub_qos", "mqtt_suback_qos", "mqtt_ver", "mqtt_willmsg", "mqtt_willmsg_len", "mqtt_willtopic", "mqtt_willtopic_len",
              "target"]
# print(len(col_names))
# print(len(col_names))
df_train = spark.read.csv(("C:/Users/intui/Desktop/Coursework/Fall 2022/(14-813A) Systems and Tool Chains in AI/Course Project/Data/FINAL_CSV/train70_augmented.csv"),header=True, inferSchema= True).toDF(*col_names)
df_train_small = df_train.limit(140000)
df_test = spark.read.csv(("C:/Users/intui/Desktop/Coursework/Fall 2022/(14-813A) Systems and Tool Chains in AI/Course Project/Data/FINAL_CSV/test30_augmented.csv"),header=True, inferSchema= True).toDF(*col_names)
df_test_small = df_test.limit(60000)



In [3]:
db_properties={}
#update your db username
db_properties['username']="postgres"
#update your db password
db_properties['password']="112358"
#make sure you got the right port number here
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
#make sure you had the Postgres JAR file in the right location
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "mqtt.MQTT_IOT"


In [4]:
df_train_small.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

df_test_small.write.format("jdbc")\
.mode("append")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

In [5]:
spark = SparkSession.builder.appName("Basics").getOrCreate()
sc=spark.sparkContext
sqlContext = SQLContext(sc)

dataf_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

C:\Users\intui\anaconda3\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
from pyspark.sql.functions import col, lit, monotonically_increasing_id, when, mean
df_index = dataf_read.select("*").withColumn("id", monotonically_increasing_id())
df_index.show(vertical = True)

-RECORD 0--------------------------------
 tcp_flags                  | 0x00000018 
 tcp_time_delta             | 1.000039   
 tcp_len                    | 13         
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0x00000030 
 mqtt_kalive                | 0.0        
 mqtt_len                   | 11.0       
 mqtt_msg                   | 32         
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 3.0        
 mqtt_proto_len             | 0.0 

In [7]:
df_index.printSchema()

root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-

In [8]:
ind = 0.7*df_index.count()
df2 = df_index.withColumn('train_test', when(df_index.id < int(ind), lit(0)).otherwise(lit(1)))

In [9]:
# ind = 0.7*dataf_read.count()
# df_index.withColumn("train_test", lit(1)).show(5, vertical = True)
# for i in range(int(ind)):
#     df_index["train_test"] == lit(0)

In [10]:
# df_trained = df_index.filter(df_index.id < int(ind))
# df_trained.count()

In [11]:
df3 = df2.filter(df2.train_test == 0)

In [12]:
df3.agg({"mqtt_len": "avg"}).show()

+------------------+
|     avg(mqtt_len)|
+------------------+
|12.161007142857143|
+------------------+



In [13]:
df2.groupby(col("target")).count().show()

+----------+-----+
|    target|count|
+----------+-----+
|   slowite|20270|
|bruteforce|20085|
|     flood|19804|
| malformed|20161|
|       dos|19788|
|legitimate|99892|
+----------+-----+



In [14]:
# attacks = np.unique(np.array(df3.select("target").collect()))
# attacks
    

In [15]:
import numpy as np
attacks = np.unique(np.array(df3.select("target").collect()))
attacks

array(['bruteforce', 'dos', 'flood', 'legitimate', 'malformed', 'slowite'],
      dtype='<U10')